# Advanced Practice: `map()`

These exercises go beyond the basics and focus on *iterator correctness*, *strict vs. permissive behavior*, and *clean functional patterns* built on top of `map()`.

👉 **Instructions**
- Implement where marked `# YOUR CODE HERE`.
- Do **not** modify test cells.
- Use only the standard library.


In [1]:
from __future__ import annotations
from typing import Iterable, Iterator, Callable, Any, Sequence, Tuple, Dict, Optional, Type
from itertools import zip_longest
import operator


## Problem 1 — `map_apply`: a minimal lazy mapper

Implement `map_apply(func, iterable)` that behaves like `map(func, iterable)` and returns a **lazy iterator** (generator). Do **not** materialize a list.

Edge cases: empty iterables; functions with side-effects should only run when the iterator is consumed.

In [2]:
def map_apply(func: Callable[[Any], Any], iterable: Iterable[Any]) -> Iterator[Any]:
    """Lazy single-iterable map (teaching re-implementation).

    Examples
    --------
    >>> list(map_apply(lambda x: x*2, [1,2,3]))
    [2, 4, 6]
    """
    for item in iterable:
        yield func(item)


In [3]:
# Tests — do not modify
side_effect = {"calls": 0}
def f(x):
    side_effect["calls"] += 1
    return x + 1

it = map_apply(f, range(3))
assert side_effect["calls"] == 0  # lazy
assert next(it) == 1 and side_effect["calls"] == 1

assert list(map_apply(lambda x: x*2, [])) == []
assert list(map_apply(lambda s: s.upper(), ["a","b"])) == ["A","B"]
print("✅ Problem 1 tests passed.")


✅ Problem 1 tests passed.


## Problem 2 — `map_strict` across *multiple* iterables

The built-in `map` stops at the *shortest* iterable. Implement `map_strict(func, *iterables)` which raises `ValueError` if the iterables have different lengths; otherwise yields `func(*xs)` lazily.

Hint: Use `zip_longest` with a unique sentinel and detect length mismatch on the fly.

In [4]:
def map_strict(func: Callable[..., Any], *iterables: Iterable[Any]) -> Iterator[Any]:
    """Lazy strict-map that errors on length mismatch.

    Examples
    --------
    >>> list(map_strict(operator.add, [1,2], [10,20]))
    [11, 22]
    """
    _MISSING = object()
    for bundle in zip_longest(*iterables, fillvalue=_MISSING):
        if _MISSING in bundle:
            raise ValueError("iterables have different lengths")
        yield func(*bundle)


In [5]:
# Tests — do not modify
assert list(map_strict(operator.mul, [1,2,3], [10,20,30])) == [10,40,90]
try:
    list(map_strict(operator.add, [1,2], [10]))
    raise AssertionError("expected ValueError for length mismatch")
except ValueError:
    pass
print("✅ Problem 2 tests passed.")


✅ Problem 2 tests passed.


## Problem 3 — `starmap_strict`

Implement `starmap_strict(func, iterable_of_args)` that behaves like `itertools.starmap`, but also ensures each element is an unpackable *Sequence* (tuple/list). If not, raise `TypeError`. Return a **lazy** iterator.


In [6]:
def starmap_strict(func: Callable[..., Any], iterable_of_args: Iterable[Sequence[Any]]) -> Iterator[Any]:
    for args in iterable_of_args:
        if not isinstance(args, (tuple, list)):
            raise TypeError("each element must be a tuple/list of arguments")
        yield func(*args)


In [7]:
# Tests — do not modify
pairs = [(1,2), (3,4), (5,6)]
assert list(starmap_strict(operator.mul, pairs)) == [2,12,30]
try:
    list(starmap_strict(operator.add, [(1,2), 3]))
    raise AssertionError("expected TypeError")
except TypeError:
    pass
print("✅ Problem 3 tests passed.")


✅ Problem 3 tests passed.


## Problem 4 — Mapping over dictionaries (`fmap_dict`)

Implement `fmap_dict(d, key_fn=None, value_fn=None)` to produce a **new dict**; if `key_fn` is provided, transform keys; if `value_fn` is provided, transform values. Both default to identity for their part.

Avoid mutating the input.

In [8]:
def fmap_dict(d: Dict[Any, Any], key_fn: Optional[Callable[[Any], Any]] = None,
              value_fn: Optional[Callable[[Any], Any]] = None) -> Dict[Any, Any]:
    kf = key_fn or (lambda x: x)
    vf = value_fn or (lambda x: x)
    return {kf(k): vf(v) for k, v in d.items()}


In [9]:
# Tests — do not modify
d = {"a": 1, "b": 2}
assert fmap_dict(d, str.upper, lambda x: x*10) == {"A": 10, "B": 20}
assert fmap_dict(d, value_fn=lambda x: (x, x)) == {"a": (1,1), "b": (2,2)}
assert d == {"a":1,"b":2}
print("✅ Problem 4 tests passed.")


✅ Problem 4 tests passed.


## Problem 5 — `map_except`: mapping with error handling

Implement `map_except(func, iterable, *, on_error='skip', default=None, exceptions=(Exception,))` that yields results of `func(x)` while handling exceptions according to `on_error`:

- `'skip'` → omit failing items
- `'default'` → yield `default` for failing items
- `'raise'` → re-raise the exception

Return a **lazy** iterator.

In [10]:
def map_except(func: Callable[[Any], Any], iterable: Iterable[Any], *,
               on_error: str = 'skip', default: Any = None,
               exceptions: Tuple[Type[BaseException], ...] = (Exception,)) -> Iterator[Any]:
    if on_error not in {'skip','default','raise'}:
        raise ValueError("on_error must be 'skip', 'default', or 'raise'")
    for item in iterable:
        try:
            yield func(item)
        except exceptions as ex:
            if on_error == 'skip':
                continue
            elif on_error == 'default':
                yield default
            else:  # 'raise'
                raise


In [11]:
# Tests — do not modify
def inv(x):
    return 1/x
data = [1, 0, 2, 0, 4]
assert list(map_except(inv, data, on_error='skip')) == [1.0, 0.5, 0.25]
assert list(map_except(inv, data, on_error='default', default=float('inf'))) == [1.0, float('inf'), 0.5, float('inf'), 0.25]
try:
    list(map_except(inv, data, on_error='raise'))
    raise AssertionError('expected ZeroDivisionError')
except ZeroDivisionError:
    pass
print("✅ Problem 5 tests passed.")


✅ Problem 5 tests passed.


## Problem 6 — `map_chunks`: process data in fixed-size chunks

Implement `map_chunks(func, iterable, chunk_size)` that groups the input into *lists of length up to `chunk_size`* (last chunk may be smaller) and yields `func(chunk_list)` for each chunk. Returns a **lazy** iterator.


In [12]:
def map_chunks(func: Callable[[Sequence[Any]], Any], iterable: Iterable[Any], chunk_size: int) -> Iterator[Any]:
    if chunk_size <= 0:
        return iter(())
    buf = []
    for x in iterable:
        buf.append(x)
        if len(buf) == chunk_size:
            yield func(buf)
            buf = []
    if buf:
        yield func(buf)


In [13]:
# Tests — do not modify
chunks = list(map_chunks(sum, range(1, 8), 3))  # [1,2,3],[4,5,6],[7]
assert chunks == [6, 15, 7]
assert list(map_chunks(len, "abcdefg", 2)) == [2,2,2,1]
assert list(map_chunks(sum, [], 4)) == []
print("✅ Problem 6 tests passed.")


✅ Problem 6 tests passed.


## Problem 7 — A clean `dot_product` using `map` with two iterables

Implement `dot_product(u, v)` (no numpy). Use `sum(map(operator.mul, u, v))`. If the lengths differ, raise `ValueError` (use `map_strict` from Problem 2 to enforce strictness).

In [14]:
def dot_product(u: Sequence[float], v: Sequence[float]) -> float:
    if len(u) != len(v):
        raise ValueError("length mismatch")
    return sum(map(operator.mul, u, v))


In [15]:
# Tests — do not modify
assert dot_product([1,2,3], [4,5,6]) == 32
try:
    dot_product([1,2], [3])
    raise AssertionError('expected ValueError')
except ValueError:
    pass
print("✅ Problem 7 tests passed.")


✅ Problem 7 tests passed.


## Problem 8 — `map_pipe`: map through a pipeline of unary functions

Implement `map_pipe(iterable, *funcs)` that applies a sequence of **unary** functions left→right to each element lazily, e.g., `map_pipe([1,2], f, g)` yields `g(f(1))`, `g(f(2))`.

If no functions are provided, return elements unchanged (identity mapping).

In [16]:
def map_pipe(iterable: Iterable[Any], *funcs: Callable[[Any], Any]) -> Iterator[Any]:
    if not funcs:
        for x in iterable:
            yield x
        return
    def apply_all(x: Any) -> Any:
        for f in funcs:
            x = f(x)
        return x
    for x in iterable:
        yield apply_all(x)


In [17]:
# Tests — do not modify
inc = lambda x: x+1
dbl = lambda x: x*2
assert list(map_pipe([1,2,3], inc, dbl)) == [4,6,8]  # (x+1)*2
assert list(map_pipe([1,2,3])) == [1,2,3]
assert list(map_pipe(["a","b"], str.upper, lambda s: f"[{s}]")) == ["[A]","[B]"]
print("✅ Problem 8 tests passed.")


✅ Problem 8 tests passed.


## Problem 9 — Laziness demonstration (evaluation happens on consumption)

Demonstrate that mapping side-effects occur only when consuming the iterator. Implement `lazy_counter(iterable)` that maps a function incrementing a shared counter for each element and yields the original elements untouched. Then show that the counter changes only as the iterator is consumed.

In [18]:
def lazy_counter(iterable: Iterable[Any]) -> Tuple[Iterator[Any], Dict[str,int]]:
    state = {"seen": 0}
    def tick(x):
        state["seen"] += 1
        return x
    return map(tick, iterable), state


In [19]:
# Tests — do not modify
it, st = lazy_counter([10,20,30])
assert st["seen"] == 0
assert next(it) == 10 and st["seen"] == 1
assert list(it) == [20,30] and st["seen"] == 3
print("✅ Problem 9 tests passed.")


✅ Problem 9 tests passed.
